[toc]

# DL-学习笔记-5-自定义LSTM层

## 代码实现

### 输入数据

先明确输入数据。 LSTM 接受的是序列数据。按照 


In [1]:
import tensorflow as tf

tf.random.set_seed(123)
batch_size = 2
n_sequences = 3
n_features = 5
units = 4
x_train = tf.random.normal(shape=(batch_size, n_sequences, n_features))

### customLSTM1

customLSTM1 是我们的一第种实现方式。直接使用上面的公式就可以。

在 tensorflow 的源码中，LSTM 的功能是由 LSTMCell 和 LSTM 两个类实现和。这个进行了简化。将它们合并成一个类。其中  lst_cell_forward 参数的功能对应于源码中 LSTMCell 的 call 方法的功能。

In [2]:
class customLSTM1(tf.keras.layers.Layer):
    def __init__(self, units, name="customLSTM1", **kwargs):
        super(customLSTM1, self).__init__(name=name, **kwargs)
        self.units = units

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.Wf = self.add_weight("Wf", shape=(self.units + input_dim, self.units))
        self.bf = self.add_weight("bf", shape=(1, self.units))
        self.Wi = self.add_weight("Wi", shape=(self.units + input_dim, self.units))
        self.bi = self.add_weight("bi", shape=(1, self.units))
        self.Wo = self.add_weight("Wo", shape=(self.units + input_dim, self.units))
        self.bo = self.add_weight("bo", shape=(1, self.units))
        self.Wc = self.add_weight("Wc", shape=(self.units + input_dim, self.units))
        self.bc = self.add_weight("bc", shape=(1, self.units))
        super(customLSTM1, self).build(input_shape)

    def lstm_cell_forward(self, xt, c_prev, h_prev):
        concat_x = tf.concat([h_prev, xt], axis=1)
        forget_gate = tf.nn.sigmoid(tf.matmul(concat_x, self.Wf) + self.bf)
        input_gate = tf.nn.sigmoid(tf.matmul(concat_x, self.Wi) + self.bi)
        output_gate = tf.nn.sigmoid(tf.matmul(concat_x, self.Wo) + self.bo)
        ctt = tf.nn.tanh(tf.matmul(concat_x, self.Wc) + self.bc)
        c_next = forget_gate * c_prev + input_gate * ctt
        h_next = output_gate * tf.nn.tanh(c_next)
        return h_next, c_next, h_next

    def call(self, x):
        batch_size, n_sequences = x.shape[0], x.shape[1]
        c_prev = tf.zeros(shape=(batch_size, self.units))
        h_prev = tf.zeros(shape=(batch_size, self.units))
        for t in range(n_sequences):
            h_next, c_next, _ = self.lstm_cell_forward(x[:, t, :], c_prev, h_prev)
            h_prev = h_prev
            c_prev = c_next
        return h_next

测试数据，可以正确输出结果。

In [3]:
mylstm1 = customLSTM1(units)
mylstm1.build(input_shape=(None, n_sequences, n_features))
mylstm1(x_train)

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[ 0.3015624 ,  0.5244602 , -0.20986232, -0.00497146],
       [ 0.33407864,  0.51461583, -0.02144831,  0.15913488]],
      dtype=float32)>

### customLSTM2

customLSTM2 是 lstm 的另一种实现。和 customLSTM2 的主要区别是将 Wr, Wi, Wo, Wc 归结为两个参数，kernel 和 recurrent_kernel。
这和 RNN 正好是对应起来的。

并且将 bf、bi、bo、bc 四个 bias 也拼接起来形成一个。这样做矩阵乘法时相当于 4 个一起来做，可以提高效率。

之后的代码我们会将 customLSTM2 的结果和 tf.keras.layers.LSTM 的结果做对比。为了两个代码可以得到相同的结果，我们需要定义一个函数来自
初始化权重。

这个函数接受 shape 和 dtype 两个参数， 返回一个 tf.Variable 对象

In [4]:
def get_weight(shape, dtype):
    tf.random.set_seed(123)
    return tf.Variable(tf.random.normal(shape=shape, dtype=dtype))

下面我们实现 `customLSTM2` 类

In [5]:
class customLSTM2(tf.keras.layers.Layer):
    def __init__(self, units,
                 name="customLSTM2",
                 use_bias=True,
                 kernel_initializer=tf.keras.initializers.glorot_normal(),
                 recurrent_initializer=tf.keras.initializers.glorot_normal(),
                 bias_initlizer=tf.keras.initializers.zeros(),
                 activation=tf.nn.tanh,
                 recurrent_activation=tf.nn.sigmoid,
                 unit_forget_bias=True, **kwargs):

        super(customLSTM2, self).__init__(name=name, **kwargs)
        self.units = units
        self.use_bias = use_bias
        self.kernel_initializer = kernel_initializer
        self.recurrent_initializer = recurrent_initializer
        self.bias_initializer = bias_initlizer
        self.unit_forget_bias = unit_forget_bias
        self.activation = activation
        self.recurrent_activation = recurrent_activation

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.kernel = self.add_weight(name="kernel", shape=(input_dim, self.units * 4), initializer=self.kernel_initializer)
        self.recurrent_kernel = self.add_weight(name="recurrent_kernel", shape=(self.units, self.units * 4), initializer=self.recurrent_initializer)
        if self.use_bias:
            if self.unit_forget_bias:
                def bias_initializer(_, dtype):
                    return tf.concat([
                        self.bias_initializer(shape=(1, self.units), dtype=dtype),
                        tf.ones(shape=(1, self.units), dtype=dtype), # forget_gate  的 bias
                        self.bias_initializer(shape=(1, self.units * 2), dtype=dtype)
                    ], axis=1)
            else:
                bias_initializer = self.bias_initializer
            self.bias = self.add_weight(name="bias", shape=(1, self.units * 4), initializer=bias_initializer)
        else:
            self.bias = None
        super(customLSTM2, self).build(input_shape)

    def lstm_cell_forward(self, xt, c_prev, h_prev):
        z = tf.matmul(xt, self.kernel)
        z += tf.matmul(h_prev, self.recurrent_kernel)
        if self.use_bias:
            z += self.bias
        z0, z1, z2, z3 = tf.split(z, 4, axis=-1)
        input_gate = self.recurrent_activation(z0)
        forget_gate = self.recurrent_activation(z1)
        ctt = self.activation(z2)
        output_gate = self.recurrent_activation(z3)
        c_next = forget_gate * c_prev + input_gate * ctt
        h_next = output_gate * self.activation(c_next)
        return h_next, c_next, h_next

    def call(self, x):
        batch_size, n_sequences = x.shape[0], x.shape[1]
        c_prev = tf.zeros(shape=(batch_size, self.units))
        h_prev = tf.zeros(shape=(batch_size, self.units))
        for t in range(n_sequences):
            h_next, c_next, _ = self.lstm_cell_forward(x[:, t, :], c_prev, h_prev)
            h_prev = h_next
            c_prev = c_next
        return h_next

上面的 `customLSTM2` 的类除了使用 kernel 和 recurrent_kernel 以及 bias 三个参数来代替原来的 8 个参数之外，没有其它本质变化。

不过，`customLSTM2` 参考了很大程度上参考了 `tf.keras.layers.LSTM`，参数名和变量名也尽可能保持一致，可以看做是 `tf.keras.layers.LSTM` 的阉割版。

其中 unit_forget_bias 参数需要特殊说明，unit_forget_bias = True 时，forget_gate 的 bias 会被初始化为 1，否则会被初始化为0。tensorflow 默认为 True。

和 tensorflow 自带的 tf.keras.layers.LSTM 做比较。结果完全相同。

In [6]:
mylstm2 = customLSTM2(units, kernel_initializer=get_weight, recurrent_initializer=get_weight)
mylstm2.build(input_shape=(None, n_sequences, n_features))

lstm = tf.keras.layers.LSTM(units, kernel_initializer=get_weight, recurrent_initializer=get_weight)
mylstm2(x_train) == lstm(x_train)

<tf.Tensor: shape=(2, 4), dtype=bool, numpy=
array([[ True,  True,  True,  True],
       [ True,  True,  True,  True]])>

实际上，customLSTM1 和 customLSTM2 在 tf.keras.layers.LSTM 类中都实现了 。tf.keras.layers.LSTM 有一个 implement 参数，就是指定实现方式。 impletement = 1 表明实现是 customLSTM1 的实现方式，否则是 customLSTM2 的实现方式。

两种实现方式没有什么本质区别。只不过 cusotmLSTM2 将四个权重放在一起计算，因此需要更多的内存，但是可能有提升效率。而 customLSTM1 会比较节省内存。